## Final - Numerical 

In [1]:
import math
import scipy.stats as ss 
import numpy as np 
import matplotlib.pyplot as plt 
plt.style.use('seaborn-whitegrid')
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(1)

In [3]:
# time index for the data with 1/100 discretization
time_index = np.arange(0.01, 1.01, 0.01)

In [4]:
# standard brownian motion covariance matrix 
bm_cov = np.reshape(np.array([i if i < j else j for j in time_index for i in time_index]), (100,100))

In [5]:
# the standard brownian motion to be implemented 
brownian_motion = np.insert(np.array([np.linalg.cholesky(bm_cov).dot(np.random.normal(loc=0, 
                                                                                      scale=1, 
                                                                                      size=100)) for _ in range(100000)]), 
                                      obj=0, values=0, axis=1)

In [6]:
brownian_motion

array([[ 0.        ,  0.16243454,  0.1012589 , ...,  0.5980254 ,
         0.53602532,  0.60582852],
       [ 0.        , -0.04471286,  0.07773791, ...,  1.3424084 ,
         1.42350357,  1.52794778],
       [ 0.        , -0.04008782,  0.04231274, ...,  0.13517043,
         0.0396279 ,  0.09821895],
       ...,
       [ 0.        , -0.12741546, -0.09444834, ..., -0.08712292,
        -0.13733226, -0.12386239],
       [ 0.        , -0.06349617, -0.05982003, ..., -1.41911905,
        -1.51724478, -1.4409592 ],
       [ 0.        , -0.09756999, -0.00791429, ..., -0.68620901,
        -0.75681393, -0.8265847 ]])